In [6]:
import networkx as nx
from collections import deque

def branch_and_bound(G):
    best_path = []
    best_cost = float('inf')

    def cost(path):
        total_cost = 0
        for i in range(len(path) - 1):
            if path[i + 1] in G[path[i]]:
                total_cost += G[path[i]][path[i + 1]]['weight']
            else:
                return float('inf')  # Retorna um custo muito alto se o caminho não existir
        return total_cost

    def explore(path):
        nonlocal best_path, best_cost
        if len(path) == len(G):
            total_cost = cost(path + [path[0]])
            if total_cost < best_cost:
                best_cost = total_cost
                best_path = path
            return

        for node in set(G.nodes) - set(path):
            explore(path + [node])

    for start_node in G.nodes:
        explore([start_node])

    return best_path, best_cost

def calcular_custo(G, path):
    cost = 0
    for i in range(len(path) - 1):
        cost += G[path[i]][path[i + 1]]['weight']
    return cost

def twice_around_the_tree(G, start_node=0):
    # Convertendo o grafo direcionado em não direcionado
    G_undirected = G.to_undirected()

    # Calculando a Árvore Geradora Mínima no grafo não direcionado
    mst = nx.minimum_spanning_tree(G_undirected)

    # Realizar a caminhada em profundidade na AGM
    visited = set()
    path = []
    stack = deque([start_node])

    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            path.append(vertex)
            stack.extend(reversed([n for n in mst.neighbors(vertex) if n not in visited]))

    # Retornar ao nó inicial para completar o circuito
    path.append(start_node)
    #cost = calcular_custo(G, path)
    return path, cost

G = nx.DiGraph()
G.add_edge(0, 1, weight=5)
G.add_edge(0, 2, weight=8)
G.add_edge(0, 3, weight=9)
G.add_edge(1, 2, weight=10)
G.add_edge(1, 3, weight=5)
G.add_edge(2, 3, weight=7)
G.add_edge(3, 0, weight=6)

path, cost = branch_and_bound(G)
print("Caminho:", path)
print("Custo:", cost)

path, cost = twice_around_the_tree(G)
print("Caminho:", path)
print("Custo:", cost)


Caminho: [0, 1, 2, 3]
Custo: 28
Caminho: [0, 1, 3, 2, 0]
Custo: 28
